# 주식리스트

1. pykrx 활용으로 그날의 거래되는 주식의 티커를 조회할 수 있음

In [6]:
import pandas as pd
from pykrx import stock

tickers_kospi = stock.get_market_ticker_list("20231203",market= 'KOSPI')
tickers_kosdaq = stock.get_market_ticker_list("20231203",market= 'KOSDAQ')

print(len(tickers_kospi),len(tickers_kosdaq))

952 1701


2. KRX에서 바로 다운로드를 받는 방법도 있음
- 주가 업데이트를 위해서는 이 방법도 나쁘지 않아보임
- 하지만 pykrx로 데이터를 받을 수 있는 시점에서 굳이 싶은 선택지

In [31]:
from io import BytesIO

def crawling_krx_data(tradingdate):
    # otp 요청 
    gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
    otp_data = {
        'locale': 'ko_KR',
        'mktId': 'ALL',
        'trdDd': tradingdate.replace('-',''),
        'share': '1',
        'money': '1',
        'csvxls_isNo': 'false',
        'name': 'fileDown',
        'url': 'dbms/MDC/STAT/standard/MDCSTAT01501'
    }
    headers = {
        'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020101',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
    }
    otp = requests.get(gen_otp_url, params = otp_data, headers=headers)
    
    # 요청받은 otp로 데이터 데이터 요청
    download_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
    download_data = {'code': otp.text}
    r = requests.get(download_url, params = download_data, headers=headers)

    df = pd.read_csv(BytesIO(r.content),encoding='euc-kr')
    df['date'] = tradingdate
    df.set_index('date',inplace=True)    
    return df

In [32]:
crawling_krx_data('2023-12-01')

,종목코드,종목명,시장구분,소속부,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수
date,,,,,,,,,,,,,,
2023-12-01,060310,3S,KOSDAQ,중견기업부,2295,15,0.66,2270,2310,2260,120193,274145660,111391593390,48536642
2023-12-01,095570,AJ네트웍스,KOSPI,NaN,4505,185,4.28,4350,4530,4305,370243,1654213940,203863679295,45252759
2023-12-01,006840,AK홀딩스,KOSPI,NaN,17740,-60,-0.34,17800,17970,17400,4225,74447290,235011732140,13247561
2023-12-01,054620,APS,KOSDAQ,중견기업부,6090,-330,-5.14,6540,6540,6030,105187,647701980,124200805890,20394221
2023-12-01,265520,AP시스템,KOSDAQ,우량기업부,21150,0,0.00,21300,21300,20800,65927,1391552000,323202054150,15281421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-01,000540,흥국화재,KOSPI,NaN,3040,55,1.84,3005,3180,2975,261882,803593015,195297640800,64242645
2023-12-01,000545,흥국화재우,KOSPI,NaN,8900,2050,29.93,6560,8900,6560,220544,1872440520,6835200000,768000
2023-12-01,003280,흥아해운,KOSPI,NaN,2260,50,2.26,2230,2425,2190,22826554,52954505340,543360271740,240424899


# 티커 베이스로 데이터 스크래핑
- historical data 업데이트 중 100 페이지 이후부터는 get request 가 불가능
- 네트워크 확인결과 referer 부분에서 유의한 차이를 보였고 이를 header에서 설정해주니 가능해짐

In [274]:
import requests
from bs4 import BeautifulSoup

# URL of the Naver Finance page you want to scrape
code = '033230'
url = f'https://finance.naver.com/item/board.naver?code={code}&page={101}'
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Safari/605.1.15',
    'Referer':'https://finance.naver.com/item/board.naver?code=001515&page=101'
}

In [275]:
response = requests.get(url,headers=headers)
soup = BeautifulSoup(response.text, 'lxml')
tbody = soup.find('tbody')

In [276]:
dates_hits = tbody.find_all('span',attrs = {'class':"tah p10 gray03"})
dates = [dates_hits[x].text.strip() for x in list(range(0,40,2))]
dates
# pd.to_datetime(pd.DataFrame(dates)[0], format='%Y-%m-%d %H:%M')

['2023.08.28 09:47',
 '2023.08.28 09:13',
 '2023.08.25 22:44',
 '2023.08.26 15:31',
 '2023.08.26 17:59',
 '2023.08.25 17:00',
 '2023.08.26 00:41',
 '2023.08.25 11:00',
 '2023.08.25 10:12',
 '2023.08.28 09:00',
 '2023.08.25 09:57',
 '2023.08.25 09:23',
 '2023.08.25 08:08',
 '2023.08.25 08:50',
 '2023.08.24 17:24',
 '2023.08.24 17:48',
 '2023.08.24 17:08',
 '2023.08.24 15:46',
 '2023.08.24 15:44',
 '2023.08.24 15:39']

In [278]:
views = [dates_hits[x].text.strip() for x in list(range(1,40,2))]
views

['344',
 '176',
 '1346',
 '689',
 '603',
 '1187',
 '664',
 '1176',
 '1783',
 '29',
 '871',
 '288',
 '705',
 '206',
 '747',
 '334',
 '750',
 '750',
 '428',
 '391']

In [279]:
link_titles = tbody.find_all('a',href = True)
links = ["https://finance.naver.com"+link['href'] for link in link_titles]
links

['https://finance.naver.com/item/board_read.naver?code=033230&nid=259089485&st=&sw=&page=101',
 'https://finance.naver.com/item/board_read.naver?code=033230&nid=259081212&st=&sw=&page=101',
 'https://finance.naver.com/item/board_read.naver?code=033230&nid=259015762&st=&sw=&page=101',
 'https://finance.naver.com/item/board_read.naver?code=033230&nid=259033064&st=&sw=&page=101',
 'https://finance.naver.com/item/board_read.naver?code=033230&nid=259035739&st=&sw=&page=101',
 'https://finance.naver.com/item/board_read.naver?code=033230&nid=259000806&st=&sw=&page=101',
 'https://finance.naver.com/item/board_read.naver?code=033230&nid=259018960&st=&sw=&page=101',
 'https://finance.naver.com/item/board_read.naver?code=033230&nid=258931353&st=&sw=&page=101',
 'https://finance.naver.com/item/board_read.naver?code=033230&nid=258919141&st=&sw=&page=101',
 'https://finance.naver.com/item/board_read.naver?code=033230&nid=259078305&st=&sw=&page=101',
 'https://finance.naver.com/item/board_read.naver?

In [280]:
titles = [title['title'] for title in link_titles]
titles

['쫌가자 인성아',
 '오늘 급등예정',
 '그냥',
 '그냥',
 '그냥',
 '원격진료 선진국이면 무조건이다',
 '[삭제된 게시물의 답글]2900원 애개리 쩌~~억!! ㅋㅋ',
 '부채비율 212% 영업적자~ 왜 이렇게 돈을 못버는거야?',
 '속보))인성정보 초대형 계약공시',
 '속보))인성정보 초대형 계약공시',
 '조졌노ㅋㅋ',
 '시간은 우리편 ㅎㅎㅎㅎㅎㅎ',
 '비대면 진료 물건너 갔음.',
 '비대면 진료 물건너 갔음.',
 '영원한 관심밖으로.',
 '영원한 관심밖으로.',
 '시외',
 '죄악 섹터',
 '인성',
 '오늘은']

In [281]:
agree_disagree = tbody.find_all('strong')
agree = [agree_disagree[x].text.strip() for x in list(range(0,40,2))]
disagree = [agree_disagree[x].text.strip() for x in list(range(1,40,2))]

In [282]:
agree

['2',
 '0',
 '8',
 '5',
 '3',
 '5',
 '4',
 '3',
 '12',
 '0',
 '7',
 '0',
 '7',
 '2',
 '5',
 '3',
 '2',
 '2',
 '1',
 '3']

In [283]:
disagree

['1',
 '1',
 '5',
 '3',
 '2',
 '1',
 '0',
 '3',
 '6',
 '0',
 '0',
 '1',
 '1',
 '3',
 '2',
 '1',
 '2',
 '2',
 '0',
 '2']

In [289]:
headers = ['date','title','id','view','like','dislike']

In [290]:
data = [[cell.text.strip() for cell in row.find_all(['td', 'th'])] for row in tbody.find_all('tr')[1:]]  # Skip the header row

In [291]:
df = pd.DataFrame(data, columns=headers)
df = df.dropna()

In [299]:
date = pd.to_datetime(df.date)

In [301]:
date

1    2023-08-28 09:47:00
2    2023-08-28 09:13:00
3    2023-08-25 22:44:00
4    2023-08-26 15:31:00
5    2023-08-26 17:59:00
7    2023-08-25 17:00:00
8    2023-08-26 00:41:00
9    2023-08-25 11:00:00
10   2023-08-25 10:12:00
11   2023-08-28 09:00:00
13   2023-08-25 09:57:00
14   2023-08-25 09:23:00
15   2023-08-25 08:08:00
16   2023-08-25 08:50:00
17   2023-08-24 17:24:00
19   2023-08-24 17:48:00
20   2023-08-24 17:08:00
21   2023-08-24 15:46:00
22   2023-08-24 15:44:00
23   2023-08-24 15:39:00
Name: date, dtype: datetime64[ns]